# MNIST 

## Importing relevant libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data preparation

In [10]:
mnist_dataset, mnist_info =tfds.load(name='mnist',with_info=True,as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

# Validation data as 10% of training data

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Scale the data

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)




Buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(Buffer_size)



validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


Batch_size = 100

train_data = train_data.batch(Batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))



## Model

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 100


model = tf.keras.Sequential([
    
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

## Optimizer and loss function

In [12]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

## Training model

In [14]:
number_epochs = 5

model.fit(train_data, epochs = number_epochs, validation_data = (validation_inputs, validation_targets), validation_steps = 1, verbose = 2)

Epoch 1/5
540/540 - 18s - loss: 0.3367 - accuracy: 0.9057 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 8s - loss: 0.1408 - accuracy: 0.9576 - val_loss: 0.1285 - val_accuracy: 0.9638
Epoch 3/5
540/540 - 8s - loss: 0.1011 - accuracy: 0.9702 - val_loss: 0.0955 - val_accuracy: 0.9705
Epoch 4/5
540/540 - 7s - loss: 0.0763 - accuracy: 0.9771 - val_loss: 0.0809 - val_accuracy: 0.9777
Epoch 5/5
540/540 - 8s - loss: 0.0624 - accuracy: 0.9809 - val_loss: 0.0671 - val_accuracy: 0.9793


## Test Model

In [15]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 2s 2s/step - loss: 0.0885 - accuracy: 0.9720


In [17]:
print('Test loss: {0: .2f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100))

Test loss:  0.09. Test accuracy:  97.20%
